In [2]:
import pandas as pd
import sqldf

In [3]:
# est. time 7-8s
dataDir='D:/Shoe data.xlsx'
salary=pd.read_excel(dataDir,sheet_name='Salary details')
prod=pd.read_excel(dataDir,sheet_name='Prod.data')
prodDetail=pd.read_excel(dataDir,sheet_name='ProductionDetail')

In [74]:
# salary.conveyor.unique()
# salary[salary.conveyor=='A-2']

In [4]:
salary[['month','year']]=''
salary['month']=salary.processdate.astype(str).str[5:7].astype(int)
salary['year']=salary.processdate.astype(str).str[:4].astype(int)
salary

,Department,conveyor,Gross,NoofEmployee,processdate,month,year
0,PED-IED,NaN,18443.2,2,2022-03-31,3,2022
1,UNIT-A LAB,A,37239.6,4,2022-08-31,8,2022
2,UNIT-A SCANNING,A,83321.6,8,2022-09-30,9,2022
3,UNIT – B FULL SHOE LINE 2,B-2,706076.4,85,2022-08-31,8,2022
4,UNIT-B BOX MAKING,NaN,56531.3,7,2022-03-31,3,2022
...,...,...,...,...,...,...,...
1164,MARKETING,NaN,35416.6,2,2022-06-30,6,2022
1165,UNIT-B PACKING QC,NaN,40736.0,1,2022-01-31,1,2022
1166,UNIT-B PRODUCTION,NaN,638651.1,27,2022-04-30,4,2022
1167,TRAINING,NaN,263726.3,6,2022-07-31,7,2022


In [5]:
salary.drop(['processdate'],axis=1).to_csv('salary.csv', index=False)

In [ ]:
# not working for unknown reason

# sqldf.run('''select *, all_salary/workers as avg_pay
#           from (
#           select sum(Gross) as all_salary, sum(NoofEmployee) as workers
#             from salary 
#           group by conveyor, month
#           ) as t''')


# sqldf.run('''
# select *, rank() over
# (partition by conveyor, month order by pairs)
# as rnk from all prod
# ''')

In [77]:
df = salary.groupby(['conveyor', 'month']).agg(
    all_salary=pd.NamedAgg(column='Gross', aggfunc='sum'),
    workers=pd.NamedAgg(column='NoofEmployee', aggfunc='sum')
).reset_index()

df['avg_pay'] = df['all_salary'] / df['workers']

df=df.sort_values(['conveyor','month'])

# get all prod.

In [199]:
prod.columns=prod.columns.str.lower()
prodCleaned=prod.rename({'customername':'client'},axis=1)
prodCleaned

,batch,client,conveyor,artno,pairs,month,year
0,190516,Sparky,C-1,A181827-Sparky,750,1,2022
1,190544,Sparky,C-1,A181827-Sparky,1507,1,2022
2,188826,Sparky,C-1,A181920-Sparky,12,1,2022
3,190518,Sparky,C-1,A181920-Sparky,484,1,2022
4,190494,Sparky,C-1,A2110169-Sparky,500,1,2022
...,...,...,...,...,...,...,...
20864,198297,Crown - P,H5-4,S2110492-Crown - P,144,7,2022
20865,198320,Crown - P,H5-4,S2110492-Crown - P,144,7,2022
20866,179777,Sparky,J-1,A211848-Sparky,1,7,2021
20867,188194,Crown FSPL USA,J-2,S2090758-Crown FSPL USA,150,3,2022


In [200]:
cols=['batch','conveyor','year','month','artno','client','pairs']
prodDetail['year']=prodDetail.chargedon.astype(str).str[:4]
prodDetail['month']=prodDetail.chargedon.astype(str).str[5:7]
prodDetailCleaned=prodDetail.rename({'customername':'client'},axis=1)[cols]
prodDetailCleaned=prodDetailCleaned[(prodDetailCleaned.year=='2022') & (prodDetailCleaned.month.isin(['09','08']))]
prodDetailCleaned

,batch,conveyor,year,month,artno,client,pairs
569,201023,B-3,2022,08,A211586-Sparky,Sparky,2550
570,201523,B-4,2022,09,A211586-Sparky,Sparky,18
833,201080,B-4,2022,08,A310112-Sparky,Sparky,1025
835,201522,B-3,2022,09,A310112-Sparky,Sparky,230
836,202199,B-4,2022,09,A310112-Sparky,Sparky,500
...,...,...,...,...,...,...,...
34567,202271,C-7,2022,09,S2390017-Crown FSPL,Crown FSPL,12
34568,202264,C-7,2022,09,S2390017-Crown FSPL,Crown FSPL,12
34587,202241,C-7,2022,09,S2390015-Crown FSPL,Crown FSPL,12
34588,202252,C-7,2022,09,S2390015-Crown FSPL,Crown FSPL,12


In [201]:
allprod=pd.concat([prodCleaned, prodDetailCleaned],ignore_index=True)

allprod[['month','year']]=allprod[['month','year']].astype(int)

allprod=allprod[allprod.year==2022]

# allprod['allps']=allprod.groupby(['conveyor','month']).pairs.sum().values

allprod

,batch,client,conveyor,artno,pairs,month,year
0,190516,Sparky,C-1,A181827-Sparky,750,1,2022
1,190544,Sparky,C-1,A181827-Sparky,1507,1,2022
2,188826,Sparky,C-1,A181920-Sparky,12,1,2022
3,190518,Sparky,C-1,A181920-Sparky,484,1,2022
4,190494,Sparky,C-1,A2110169-Sparky,500,1,2022
...,...,...,...,...,...,...,...
22274,202271,Crown FSPL,C-7,S2390017-Crown FSPL,12,9,2022
22275,202264,Crown FSPL,C-7,S2390017-Crown FSPL,12,9,2022
22276,202241,Crown FSPL,C-7,S2390015-Crown FSPL,12,9,2022
22277,202252,Crown FSPL,C-7,S2390015-Crown FSPL,12,9,2022


In [202]:
allprod['rnk'] = allprod.groupby(['conveyor', 'month'])['pairs'].rank(method='min', ascending=0)

top5=allprod[allprod.rnk.isin(range(1,6))].sort_values(['conveyor','month','rnk'])
top5

,batch,client,conveyor,artno,pairs,month,year,rnk
13252,185026,Rhino,A-1,S160462-Rhino,8652,1,2022,1.0
13279,181336,Bull-A,A-1,S201207-Bull -D,7417,1,2022,2.0
13298,185837,Curfu,A-1,A2210150-Curfo-A,5651,1,2022,3.0
13253,188390,Rhino,A-1,S160462-Rhino,4546,1,2022,4.0
13297,185836,Curfu,A-1,A2210150-Curfo-A,4195,1,2022,5.0
...,...,...,...,...,...,...,...,...
21997,195970,KITO-Palms,P-4,S2290287-KITO-Palms,8460,9,2022,1.0
21991,182118,KITO-Palms,P-4,A2110339-KITO - C,3174,9,2022,2.0
21994,202305,KITO-Palms,P-4,A2110339-KITO - C,270,9,2022,3.0
21990,202308,KITO-Palms,P-4,A2110339-KITO - C,228,9,2022,4.0


In [203]:
for i in range(1,6):
    var1='cus'+str(i)
    var2=var1+'ps'
    df[var1]=''
    df[var2]=''

df['allps']=0

In [206]:
# allConveyors=allprod.conveyor.unique()

for i,j in df.iterrows():
    df.loc[i,'allps']=allprod[(allprod.conveyor==j.conveyor) & (allprod.month==j.month)].pairs.sum()
    for r in range(1,6):
        var1='cus'+str(r)
        var2=var1+'ps'
        match=top5[(top5.conveyor==j.conveyor) & (top5.month==j.month)]
        try:
            df.loc[i, var1]=match.iloc[r-1]['client']
            df.loc[i, var2]=match.iloc[r-1]['pairs']
        except:
            df.loc[i, var1]=''
            df.loc[i, var2]=''

In [209]:
# ok, only A and C-6 @ aug no data:

df[df.cus1=='']

,conveyor,month,all_salary,workers,avg_pay,cus1,cus1ps,cus2,cus2ps,cus3,cus3ps,cus4,cus4ps,cus5,cus5ps,allps
0,A,1,3523429.3,252,13981.862302,,,,,,,,,,,0
1,A,2,3344768.1,232,14417.103879,,,,,,,,,,,0
2,A,3,3279022.6,229,14318.875983,,,,,,,,,,,0
3,A,4,3496435.4,245,14271.164898,,,,,,,,,,,0
4,A,5,3385860.8,244,13876.478689,,,,,,,,,,,0
5,A,6,3505505.3,248,14135.102016,,,,,,,,,,,0
6,A,7,3374496.9,244,13829.905328,,,,,,,,,,,0
7,A,8,3396375.9,246,13806.406098,,,,,,,,,,,0
8,A,9,3621236.6,254,14256.837008,,,,,,,,,,,0
133,C-6,8,352655.0,46,7666.413043,,,,,,,,,,,0


In [212]:
# allprod[(allprod.conveyor=='C-7') & (allprod.month==6)]
df

,conveyor,month,all_salary,workers,avg_pay,cus1,cus1ps,cus2,cus2ps,cus3,cus3ps,cus4,cus4ps,cus5,cus5ps,allps
0,A,1,3523429.3,252,13981.862302,,,,,,,,,,,0
1,A,2,3344768.1,232,14417.103879,,,,,,,,,,,0
2,A,3,3279022.6,229,14318.875983,,,,,,,,,,,0
3,A,4,3496435.4,245,14271.164898,,,,,,,,,,,0
4,A,5,3385860.8,244,13876.478689,,,,,,,,,,,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,C-6,9,298253.1,38,7848.765789,Crown FSPL,685,,,,,,,,,685
135,C-7,6,358132.7,48,7461.097917,Crown FSPL USA,5501,Crown FSPL - C2,980,Crown FSPL,764,Crown FSPL,603,Crown FSPL,536,10329
136,C-7,7,384174.2,49,7840.289796,Crown FSPL - C2,2598,Crown FSPL - C2,1632,Crown FSPL - C2,1008,Crown FSPL - C2,956,Crown FSPL - C2,720,8591
137,C-7,8,400679.3,52,7705.371154,Crown FSPL - C2,504,Crown FSPL USA - C2,504,Crown FSPL USA - C2,504,Sparky,500,Sparky,500,4044


In [213]:
df.to_csv('shoes.csv', index=0)